In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import TimeDistributed, Conv1D, MaxPooling1D,Flatten, LSTM, Reshape, UpSampling1D,Dropout, Dense, AveragePooling1D 
from sklearn.metrics import classification_report
from tensorflow import keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_X=np.load('/content/drive/MyDrive/pos/proj/model_datasets/train_in_array.npy')
train_Y=np.load('/content/drive/MyDrive/pos/proj/model_datasets/train_out_array.npy')

valid_X=np.load('/content/drive/MyDrive/pos/proj/model_datasets/valid_in_array.npy')
valid_Y=np.load('/content/drive/MyDrive/pos/proj/model_datasets/valid_out_array.npy')

test_X=np.load('/content/drive/MyDrive/pos/proj/model_datasets/test_in_array.npy')
test_Y=np.load('/content/drive/MyDrive/pos/proj/model_datasets/test_out_array.npy')


In [ ]:
print(train_X.shape)
print(train_Y.shape)
print(valid_X.shape)
print(valid_Y.shape)
print(test_X.shape)
print(test_Y.shape)

(887, 64, 98)
(887, 64, 6)
(104, 64, 98)
(104, 64, 6)
(105, 64, 98)
(105, 64, 6)


In [ ]:
valid_X=tf.convert_to_tensor(valid_X[None,:,:,:])
valid_Y=tf.convert_to_tensor(valid_Y[None,:,:,:])


Primeiro modelo

2 camadas convolucionais 16 e 32 filtros com kernel 3 e pooling de tamanho 2

Dropout de 0,2

Camada LSTM de 64 unidades, dropoutr recorrente de 0,5

Camada convolucional de 1 filtro, kernel, função softmax para classificação

learning rate de 1e-3

In [ ]:
steps=104
fips=64
features=98
classes=6

modelo = Sequential()
#CNN
modelo.add(TimeDistributed(Conv1D(16,(3),activation='relu',padding='same'),
                                  input_shape=(steps,fips,features)))
modelo.add(TimeDistributed(MaxPooling1D(pool_size=(2))))
modelo.add(TimeDistributed(Conv1D(32,(3),activation='relu',padding='same'),
                                  input_shape=(steps,fips,features)))
modelo.add(TimeDistributed(MaxPooling1D(pool_size=(2))))
modelo.add(Dropout(0.2))
modelo.add(TimeDistributed(Flatten()))

#LSTM
modelo.add(LSTM(units=64, return_sequences=True,recurrent_dropout=0.5))

#Softmax
modelo.add(TimeDistributed(Reshape((64, 1))))
modelo.add(TimeDistributed(Conv1D(classes, (1), activation='softmax', padding='same')))

modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 104, 64, 16)      4720      
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 104, 32, 16)      0         
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 104, 32, 32)      1568      
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 16, 32)      0         
 tributed)                                                       
                                                                 
 dropout (Dropout)           (None, 104, 16, 32)       0

Treinamento será realizado várias utilizando steps de 104 pois esse é número de steps no data set de validação 

In [ ]:
for step in [(0,104),(104,208),(312,416),(520,624),(624,728),(728,832),(783,887)]:
  train_X_step=train_X[step[0]:step[1]]
  train_Y_step=train_Y[step[0]:step[1]]
  train_X_tensor=tf.convert_to_tensor(train_X_step[None,:,:,:])
  train_Y_tensor=tf.convert_to_tensor(train_Y_step[None,:,:,:])  
  modelo.fit(train_X_tensor,train_Y_tensor,epochs=100,validation_data=(valid_X,valid_Y))

Epoch 1/100
1/1 [==============================] - 14s 14s/step - loss: 1.8018 - accuracy: 0.0233 - val_loss: 1.7984 - val_accuracy: 0.1083
Epoch 2/100
1/1 [==============================] - 1s 675ms/step - loss: 1.7938 - accuracy: 0.0284 - val_loss: 1.8014 - val_accuracy: 0.1046
Epoch 3/100
1/1 [==============================] - 1s 710ms/step - loss: 1.7885 - accuracy: 0.0352 - val_loss: 1.8043 - val_accuracy: 0.0921
Epoch 4/100
1/1 [==============================] - 1s 660ms/step - loss: 1.7852 - accuracy: 0.0404 - val_loss: 1.8066 - val_accuracy: 0.0768
Epoch 5/100
1/1 [==============================] - 1s 669ms/step - loss: 1.7822 - accuracy: 0.0479 - val_loss: 1.8083 - val_accuracy: 0.0687
Epoch 6/100
1/1 [==============================] - 1s 698ms/step - loss: 1.7795 - accuracy: 0.0512 - val_loss: 1.8097 - val_accuracy: 0.0669
Epoch 7/100
1/1 [==============================] - 1s 713ms/step - loss: 1.7772 - accuracy: 0.0512 - val_loss: 1.8109 - val_accuracy: 0.0636
Epoch 8/100
1/

Este primeiro modelo, terminou com o acurácia de 39% no dataser de validação, não é uma acurácia muito alta e bem abaixo da acurácia do teste, 69%. Acredito que um aumento do dropout pode ajudar a aproximar a acúracia de valdação à de teste

Segundo modelo

2 camadas convolucionais 16 e 32 filtros com kernel 5 e 3 e pooling de tamanho 4 e 2

Dropout de 0,3

Camada LSTM de 64 unidades, dropoutr recorrente de 0,8

Camada convolucional de 1 filtro, kernel, função softmax para classificação

learning rate de 1e-3


In [ ]:
steps=104
fips=64
features=98
classes=6

modelo = Sequential()
#CNN
modelo.add(TimeDistributed(Conv1D(16,(5),activation='relu',padding='same'),
                                  input_shape=(steps,fips,features)))
modelo.add(TimeDistributed(MaxPooling1D(pool_size=(4))))
modelo.add(TimeDistributed(Conv1D(32,(3),activation='relu',padding='same'),
                                  input_shape=(steps,fips,features)))
modelo.add(TimeDistributed(MaxPooling1D(pool_size=(2))))
modelo.add(Dropout(0.3))
modelo.add(TimeDistributed(Flatten()))

#LSTM
modelo.add(LSTM(units=64, return_sequences=True,recurrent_dropout=0.8))

#Softmax
modelo.add(TimeDistributed(Reshape((64, 1))))
modelo.add(TimeDistributed(Conv1D(classes, (1), activation='softmax', padding='same')))

modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
modelo.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_14 (TimeDi  (None, 104, 64, 16)      7856      
 stributed)                                                      
                                                                 
 time_distributed_15 (TimeDi  (None, 104, 16, 16)      0         
 stributed)                                                      
                                                                 
 time_distributed_16 (TimeDi  (None, 104, 16, 32)      1568      
 stributed)                                                      
                                                                 
 time_distributed_17 (TimeDi  (None, 104, 8, 32)       0         
 stributed)                                                      
                                                                 
 dropout_2 (Dropout)         (None, 104, 8, 32)       

In [ ]:
for step in [(0,104),(104,208),(312,416),(520,624),(624,728),(728,832),(783,887)]:
  train_X_step=train_X[step[0]:step[1]]
  train_Y_step=train_Y[step[0]:step[1]]
  train_X_tensor=tf.convert_to_tensor(train_X_step[None,:,:,:])
  train_Y_tensor=tf.convert_to_tensor(train_Y_step[None,:,:,:])  
  modelo.fit(train_X_tensor,train_Y_tensor,epochs=100,validation_data=(valid_X,valid_Y))

Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 1.7974 - accuracy: 0.0234 - val_loss: 1.7870 - val_accuracy: 0.1264
Epoch 2/100
1/1 [==============================] - 1s 691ms/step - loss: 1.7819 - accuracy: 0.0361 - val_loss: 1.7802 - val_accuracy: 0.1242
Epoch 3/100
1/1 [==============================] - 1s 645ms/step - loss: 1.7679 - accuracy: 0.0515 - val_loss: 1.7745 - val_accuracy: 0.1188
Epoch 4/100
1/1 [==============================] - 1s 673ms/step - loss: 1.7528 - accuracy: 0.0562 - val_loss: 1.7699 - val_accuracy: 0.1232
Epoch 5/100
1/1 [==============================] - 1s 654ms/step - loss: 1.7401 - accuracy: 0.0524 - val_loss: 1.7664 - val_accuracy: 0.1264
Epoch 6/100
1/1 [==============================] - 1s 653ms/step - loss: 1.7274 - accuracy: 0.0718 - val_loss: 1.7640 - val_accuracy: 0.1291
Epoch 7/100
1/1 [==============================] - 1s 691ms/step - loss: 1.7144 - accuracy: 0.0711 - val_loss: 1.7627 - val_accuracy: 0.1336
Epoch 8/100
1/1 

Desempenho semelhante ao modelo anterior

Terceiro modelo

2 camadas convolucionais 16 e 32 filtros com kernel 5 e 3 e pooling de tamanho 3 e 2

Dropout de 0,2

2 camada LSTM de 64 unidades, dropoutr recorrente de 0,5

Camada convolucional de 1 filtro, kernel, função softmax para classificação

learning rate de 1e-3


In [ ]:
steps=104
fips=64
features=98
classes=6

modelo = Sequential()
#CNN
modelo.add(TimeDistributed(Conv1D(16,(5),activation='relu',padding='same'),
                                  input_shape=(steps,fips,features)))
modelo.add(TimeDistributed(MaxPooling1D(pool_size=(3))))
modelo.add(TimeDistributed(Conv1D(32,(3),activation='relu',padding='same'),
                                  input_shape=(steps,fips,features)))
modelo.add(TimeDistributed(MaxPooling1D(pool_size=(2))))
modelo.add(Dropout(0.2))
modelo.add(TimeDistributed(Flatten()))

#LSTM
modelo.add(LSTM(units=64, return_sequences=True,recurrent_dropout=0.5))
modelo.add(LSTM(units=64, return_sequences=True,recurrent_dropout=0.5))

#Softmax
modelo.add(TimeDistributed(Reshape((64, 1))))
modelo.add(TimeDistributed(Conv1D(classes, (1), activation='softmax', padding='same')))


modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
modelo.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_28 (TimeDi  (None, 104, 64, 16)      7856      
 stributed)                                                      
                                                                 
 time_distributed_29 (TimeDi  (None, 104, 21, 16)      0         
 stributed)                                                      
                                                                 
 time_distributed_30 (TimeDi  (None, 104, 21, 32)      1568      
 stributed)                                                      
                                                                 
 time_distributed_31 (TimeDi  (None, 104, 10, 32)      0         
 stributed)                                                      
                                                                 
 dropout_4 (Dropout)         (None, 104, 10, 32)      

In [ ]:
for step in [(0,104),(104,208),(312,416),(520,624),(624,728),(728,832),(783,887)]:
  train_X_step=train_X[step[0]:step[1]]
  train_Y_step=train_Y[step[0]:step[1]]
  train_X_tensor=tf.convert_to_tensor(train_X_step[None,:,:,:])
  train_Y_tensor=tf.convert_to_tensor(train_Y_step[None,:,:,:])  
  modelo.fit(train_X_tensor,train_Y_tensor,epochs=100,validation_data=(valid_X,valid_Y))

Epoch 1/100
1/1 [==============================] - 7s 7s/step - loss: 1.7986 - accuracy: 0.1146 - val_loss: 1.7856 - val_accuracy: 0.1821
Epoch 2/100
1/1 [==============================] - 1s 1s/step - loss: 1.7859 - accuracy: 0.1767 - val_loss: 1.7851 - val_accuracy: 0.1899
Epoch 3/100
1/1 [==============================] - 2s 2s/step - loss: 1.7785 - accuracy: 0.2251 - val_loss: 1.7853 - val_accuracy: 0.1931
Epoch 4/100
1/1 [==============================] - 1s 1s/step - loss: 1.7696 - accuracy: 0.2614 - val_loss: 1.7851 - val_accuracy: 0.2060
Epoch 5/100
1/1 [==============================] - 1s 1s/step - loss: 1.7632 - accuracy: 0.2701 - val_loss: 1.7860 - val_accuracy: 0.2075
Epoch 6/100
1/1 [==============================] - 1s 1s/step - loss: 1.7560 - accuracy: 0.3026 - val_loss: 1.7876 - val_accuracy: 0.2130
Epoch 7/100
1/1 [==============================] - 1s 1s/step - loss: 1.7510 - accuracy: 0.3099 - val_loss: 1.7891 - val_accuracy: 0.2033
Epoch 8/100
1/1 [=================

Terceiro modelo

2 camadas convolucionais 16 e 32 filtros com kernel 5 e 3 e pooling de tamanho 3 e 2

Dropout de 0,2

2 camada LSTM de 64 unidades, dropoutr recorrente de 0,5

Camada convolucional de 1 filtro, kernel 1, função softmax para classificação

learning rate de 5e-4


In [ ]:
steps=104
fips=64
features=98
classes=6

modelo = Sequential()
#CNN
modelo.add(TimeDistributed(Conv1D(16,(5),activation='relu',padding='same'),
                                  input_shape=(steps,fips,features)))
modelo.add(TimeDistributed(MaxPooling1D(pool_size=(3))))
modelo.add(TimeDistributed(Conv1D(32,(3),activation='relu',padding='same'),
                                  input_shape=(steps,fips,features)))
modelo.add(TimeDistributed(MaxPooling1D(pool_size=(2))))

modelo.add(Dropout(0.2))
modelo.add(TimeDistributed(Flatten()))

#LSTM
modelo.add(LSTM(units=64, return_sequences=True,recurrent_dropout=0.5))

#Softmax
modelo.add(TimeDistributed(Reshape((64, 1))))
#modelo.add(TimeDistributed(UpSampling1D((2))))
modelo.add(TimeDistributed(Conv1D(classes, (1), activation='softmax', padding='same')))


modelo.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(5e-4), metrics=['accuracy'])
modelo.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_42 (TimeDi  (None, 104, 64, 16)      7856      
 stributed)                                                      
                                                                 
 time_distributed_43 (TimeDi  (None, 104, 21, 16)      0         
 stributed)                                                      
                                                                 
 time_distributed_44 (TimeDi  (None, 104, 21, 32)      1568      
 stributed)                                                      
                                                                 
 time_distributed_45 (TimeDi  (None, 104, 10, 32)      0         
 stributed)                                                      
                                                                 
 dropout_6 (Dropout)         (None, 104, 10, 32)      

In [ ]:
for step in [(0,104),(104,208),(312,416),(520,624),(624,728),(728,832),(783,887)]:
  train_X_step=train_X[step[0]:step[1]]
  train_Y_step=train_Y[step[0]:step[1]]
  train_X_tensor=tf.convert_to_tensor(train_X_step[None,:,:,:])
  train_Y_tensor=tf.convert_to_tensor(train_Y_step[None,:,:,:])  
  modelo.fit(train_X_tensor,train_Y_tensor,epochs=100,validation_data=(valid_X,valid_Y))

Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 1.8070 - accuracy: 0.3323 - val_loss: 1.7950 - val_accuracy: 0.2389
Epoch 2/100
1/1 [==============================] - 1s 710ms/step - loss: 1.7835 - accuracy: 0.3663 - val_loss: 1.7868 - val_accuracy: 0.2554
Epoch 3/100
1/1 [==============================] - 1s 697ms/step - loss: 1.7650 - accuracy: 0.3978 - val_loss: 1.7793 - val_accuracy: 0.2704
Epoch 4/100
1/1 [==============================] - 1s 816ms/step - loss: 1.7459 - accuracy: 0.4253 - val_loss: 1.7723 - val_accuracy: 0.2858
Epoch 5/100
1/1 [==============================] - 1s 692ms/step - loss: 1.7273 - accuracy: 0.4573 - val_loss: 1.7653 - val_accuracy: 0.2976
Epoch 6/100
1/1 [==============================] - 1s 791ms/step - loss: 1.7068 - accuracy: 0.4883 - val_loss: 1.7584 - val_accuracy: 0.3157
Epoch 7/100
1/1 [==============================] - 1s 791ms/step - loss: 1.6922 - accuracy: 0.5108 - val_loss: 1.7519 - val_accuracy: 0.3260
Epoch 8/100
1/1 

Quarto modelo

3 camadas convolucionais 16, 32 e 32 filtros com kernel 5 e 3,3 e pooling de tamanho 3, 2,2

Dropout de 0,2

2 camada LSTM de 64 unidades, dropoutr recorrente de 0,5

Camada convolucional de 1 filtro, kernel 1, função softmax para classificação

learning rate de 5e-4


In [ ]:
steps=104
fips=64
features=98
classes=6

modelo = Sequential()
#CNN
modelo.add(TimeDistributed(Conv1D(16,(5),activation='relu',padding='same'),
                                  input_shape=(steps,fips,features)))
modelo.add(TimeDistributed(MaxPooling1D(pool_size=(3))))
modelo.add(TimeDistributed(Conv1D(32,(3),activation='relu',padding='same'),
                                  input_shape=(steps,fips,features)))
modelo.add(TimeDistributed(MaxPooling1D(pool_size=(2))))
modelo.add(TimeDistributed(Conv1D(32,(3),activation='relu',padding='same'),
                                  input_shape=(steps,fips,features)))
modelo.add(TimeDistributed(MaxPooling1D(pool_size=(2))))

modelo.add(Dropout(0.2))
modelo.add(TimeDistributed(Flatten()))

#LSTM
modelo.add(LSTM(units=64, return_sequences=True,recurrent_dropout=0.5))

#Softmax
modelo.add(TimeDistributed(Reshape((64, 1))))
#modelo.add(TimeDistributed(UpSampling1D((2))))
modelo.add(TimeDistributed(Conv1D(classes, (1), activation='softmax', padding='same')))


modelo.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(5e-4), metrics=['accuracy'])
modelo.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_58 (TimeDi  (None, 104, 64, 16)      7856      
 stributed)                                                      
                                                                 
 time_distributed_59 (TimeDi  (None, 104, 21, 16)      0         
 stributed)                                                      
                                                                 
 time_distributed_60 (TimeDi  (None, 104, 21, 32)      1568      
 stributed)                                                      
                                                                 
 time_distributed_61 (TimeDi  (None, 104, 10, 32)      0         
 stributed)                                                      
                                                                 
 time_distributed_62 (TimeDi  (None, 104, 10, 32)     

In [ ]:
for step in [(0,104),(104,208),(312,416),(520,624),(624,728),(728,832),(783,887)]:
  train_X_step=train_X[step[0]:step[1]]
  train_Y_step=train_Y[step[0]:step[1]]
  train_X_tensor=tf.convert_to_tensor(train_X_step[None,:,:,:])
  train_Y_tensor=tf.convert_to_tensor(train_Y_step[None,:,:,:])  
  modelo.fit(train_X_tensor,train_Y_tensor,epochs=100,validation_data=(valid_X,valid_Y))

Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 1.8051 - accuracy: 0.3364 - val_loss: 1.7886 - val_accuracy: 0.2234
Epoch 2/100
1/1 [==============================] - 1s 643ms/step - loss: 1.7721 - accuracy: 0.3852 - val_loss: 1.7777 - val_accuracy: 0.2605
Epoch 3/100
1/1 [==============================] - 1s 753ms/step - loss: 1.7466 - accuracy: 0.4324 - val_loss: 1.7686 - val_accuracy: 0.2809
Epoch 4/100
1/1 [==============================] - 1s 724ms/step - loss: 1.7209 - accuracy: 0.4662 - val_loss: 1.7610 - val_accuracy: 0.3027
Epoch 5/100
1/1 [==============================] - 1s 701ms/step - loss: 1.6989 - accuracy: 0.5017 - val_loss: 1.7551 - val_accuracy: 0.3161
Epoch 6/100
1/1 [==============================] - 1s 735ms/step - loss: 1.6801 - accuracy: 0.5176 - val_loss: 1.7502 - val_accuracy: 0.3226
Epoch 7/100
1/1 [==============================] - 1s 695ms/step - loss: 1.6609 - accuracy: 0.5334 - val_loss: 1.7458 - val_accuracy: 0.3290
Epoch 8/100
1/1 

Quinto modelo

2 camadas convolucionais 16 e 32 filtros com kernel 5 e 3 e pooling de tamanho 3, 2

Dropout de 0,2

2 camada LSTM de 64 unidades, dropoutr recorrente de 0,2

Camada convolucional de 1 filtro, kernel 1, função softmax para classificação

learning rate de 5e-4, 1000 épocas


In [ ]:
steps=104
fips=64
features=98
classes=6

modelo = Sequential()
#CNN
modelo.add(TimeDistributed(Conv1D(16,(5),activation='relu',padding='same'),
                                  input_shape=(steps,fips,features)))
modelo.add(TimeDistributed(MaxPooling1D(pool_size=(3))))
modelo.add(TimeDistributed(Conv1D(32,(3),activation='relu',padding='same'),
                                  input_shape=(steps,fips,features)))
modelo.add(TimeDistributed(MaxPooling1D(pool_size=(2))))
modelo.add(Dropout(0.2))
modelo.add(TimeDistributed(Flatten()))

#LSTM
modelo.add(LSTM(units=64, return_sequences=True,recurrent_dropout=0.2))

#Softmax
modelo.add(TimeDistributed(Reshape((64, 1))))
modelo.add(TimeDistributed(Conv1D(classes, (1), activation='softmax', padding='same')))

modelo.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(5e-4), metrics=['accuracy'])
modelo.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_67 (TimeDi  (None, 104, 64, 16)      7856      
 stributed)                                                      
                                                                 
 time_distributed_68 (TimeDi  (None, 104, 21, 16)      0         
 stributed)                                                      
                                                                 
 time_distributed_69 (TimeDi  (None, 104, 21, 32)      1568      
 stributed)                                                      
                                                                 
 time_distributed_70 (TimeDi  (None, 104, 10, 32)      0         
 stributed)                                                      
                                                                 
 dropout_9 (Dropout)         (None, 104, 10, 32)      

In [ ]:
for step in [(0,104),(104,208),(312,416),(520,624),(624,728),(728,832),(783,887)]:
  train_X_step=train_X[step[0]:step[1]]
  train_Y_step=train_Y[step[0]:step[1]]
  train_X_tensor=tf.convert_to_tensor(train_X_step[None,:,:,:])
  train_Y_tensor=tf.convert_to_tensor(train_Y_step[None,:,:,:])  
  modelo.fit(train_X_tensor,train_Y_tensor,epochs=1000,validation_data=(valid_X,valid_Y))

A saída de streaming foi truncada nas últimas 5000 linhas.
1/1 [==============================] - 1s 758ms/step - loss: 1.3500 - accuracy: 0.3513 - val_loss: 1.8179 - val_accuracy: 0.2964
Epoch 502/1000
1/1 [==============================] - 1s 868ms/step - loss: 1.3513 - accuracy: 0.3507 - val_loss: 1.8222 - val_accuracy: 0.2942
Epoch 503/1000
1/1 [==============================] - 1s 904ms/step - loss: 1.3497 - accuracy: 0.3510 - val_loss: 1.8329 - val_accuracy: 0.2907
Epoch 504/1000
1/1 [==============================] - 1s 861ms/step - loss: 1.3490 - accuracy: 0.3504 - val_loss: 1.8435 - val_accuracy: 0.2873
Epoch 505/1000
1/1 [==============================] - 1s 750ms/step - loss: 1.3485 - accuracy: 0.3513 - val_loss: 1.8500 - val_accuracy: 0.2834
Epoch 506/1000
1/1 [==============================] - 1s 766ms/step - loss: 1.3485 - accuracy: 0.3511 - val_loss: 1.8492 - val_accuracy: 0.2847
Epoch 507/1000
1/1 [==============================] - 1s 766ms/step - loss: 1.3482 - accurac

O auemnto de epoch não resultou em um melhor treinamento

Sexto modelo

2 camadas convolucionais 16 e 32 filtros com kernel 5 e 3 e pooling de tamanho 3, 2

Dropout de 0,2

2 camada LSTM de 64 unidades, dropoutr recorrente de 0,2

Camada convolucional de 1 filtro, kernel 1, função softmax para classificação

learning rate de 5e-4, 100 épocas


In [ ]:
steps=104
fips=64
features=98
classes=6

modelo = Sequential()
#CNN
modelo.add(TimeDistributed(Conv1D(16,(5),activation='relu',padding='same'),
                                  input_shape=(steps,fips,features)))
modelo.add(TimeDistributed(AveragePooling1D(pool_size=(3))))
modelo.add(TimeDistributed(Conv1D(32,(3),activation='relu',padding='same'),
                                  input_shape=(steps,fips,features)))
modelo.add(TimeDistributed(AveragePooling1D(pool_size=(2))))

modelo.add(Dropout(0.2))
modelo.add(TimeDistributed(Flatten()))

#LSTM
modelo.add(LSTM(units=64, return_sequences=True,recurrent_dropout=0.2))
modelo.add(LSTM(units=64, return_sequences=True,recurrent_dropout=0.2))

#Softmax
modelo.add(TimeDistributed(Reshape((64, 1))))
modelo.add(TimeDistributed(Conv1D(classes, (1), activation='softmax', padding='same')))

modelo.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(5e-4), metrics=['accuracy'])
modelo.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_85 (TimeDi  (None, 104, 64, 16)      7856      
 stributed)                                                      
                                                                 
 time_distributed_86 (TimeDi  (None, 104, 21, 16)      0         
 stributed)                                                      
                                                                 
 time_distributed_87 (TimeDi  (None, 104, 21, 32)      1568      
 stributed)                                                      
                                                                 
 time_distributed_88 (TimeDi  (None, 104, 10, 32)      0         
 stributed)                                                      
                                                                 
 dropout_11 (Dropout)        (None, 104, 10, 32)     

In [ ]:
for step in [(0,104),(104,208),(312,416),(520,624),(624,728),(728,832),(783,887)]:
  train_X_step=train_X[step[0]:step[1]]
  train_Y_step=train_Y[step[0]:step[1]]
  train_X_tensor=tf.convert_to_tensor(train_X_step[None,:,:,:])
  train_Y_tensor=tf.convert_to_tensor(train_Y_step[None,:,:,:])  
  modelo.fit(train_X_tensor,train_Y_tensor,epochs=100,validation_data=(valid_X,valid_Y))

Epoch 1/100
1/1 [==============================] - 7s 7s/step - loss: 1.7946 - accuracy: 0.3104 - val_loss: 1.7905 - val_accuracy: 0.3003
Epoch 2/100
1/1 [==============================] - 1s 1s/step - loss: 1.7892 - accuracy: 0.4398 - val_loss: 1.7879 - val_accuracy: 0.3374
Epoch 3/100
1/1 [==============================] - 1s 1s/step - loss: 1.7842 - accuracy: 0.5230 - val_loss: 1.7856 - val_accuracy: 0.3478
Epoch 4/100
1/1 [==============================] - 1s 1s/step - loss: 1.7797 - accuracy: 0.5598 - val_loss: 1.7836 - val_accuracy: 0.3431
Epoch 5/100
1/1 [==============================] - 1s 1s/step - loss: 1.7744 - accuracy: 0.5760 - val_loss: 1.7815 - val_accuracy: 0.3471
Epoch 6/100
1/1 [==============================] - 1s 1s/step - loss: 1.7698 - accuracy: 0.5938 - val_loss: 1.7794 - val_accuracy: 0.3529
Epoch 7/100
1/1 [==============================] - 1s 1s/step - loss: 1.7645 - accuracy: 0.5996 - val_loss: 1.7774 - val_accuracy: 0.3562
Epoch 8/100
1/1 [=================

In [ ]:
modelo.save('/content/drive/MyDrive/pos/proj/model_datasets')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/pos/proj/model_datasets/assets


In [ ]:
modelo = keras.models.load_model('/content/drive/MyDrive/pos/proj/model_datasets')

In [ ]:
#Forecast do dataset de test
test_X=test_X[:-1]
test_X=tf.convert_to_tensor(test_X[None,:,:,:])
forecasting=modelo.predict(test_X)

In [ ]:
forecasting=forecasting[0].reshape(6656,6)

In [ ]:
#transformando probabiidade em classes
forecasting_class=[]
for lista in forecasting:
  classes=[1 if valor==max(lista) else 0 for valor in lista ]
  forecasting_class.append(classes)


In [ ]:
y=test_Y[:-1].reshape(6656,6)

In [ ]:
print(classification_report(y,forecasting_class))

              precision    recall  f1-score   support

           0       0.35      1.00      0.52      2354
           1       0.00      0.00      0.00      1020
           2       0.00      0.00      0.00      1078
           3       0.00      0.00      0.00      1057
           4       0.00      0.00      0.00       906
           5       0.00      0.00      0.00       241

   micro avg       0.35      0.35      0.35      6656
   macro avg       0.06      0.17      0.09      6656
weighted avg       0.13      0.35      0.18      6656
 samples avg       0.35      0.35      0.35      6656



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
